In [1]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [2]:
spark = SparkSession.builder \
    .enableHiveSupport().getOrCreate()
df = spark.read.options(header='True', inferSchema='True', delimiter=',') \
  .csv("final_data.csv")

In [3]:
df_assembler = VectorAssembler(inputCols=[ 'total_loan','year_of_loan','interest','monthly_payment','class', 'sub_class', 'work_type', 'employer_type', 'industry',
 'work_year',
 'house_exist',
 'house_loan_status',
 'censor_status',
 'marriage',
 'offsprings',
 'use',
 'post_code',
 'region',
 'debt_loan_ratio',
 'del_in_18month',
 'scoring_low',
 'scoring_high',
 'pub_dero_bankrup',
 'early_return',
 'early_return_amount',
 'early_return_amount_3mon',
 'recircle_b',
 'recircle_u',
 'initial_list_status',
 'title',
 'policy_code',
 'f0',
 'f1',
 'f2',
 'f3',
 'f4',
 'f5',
 'total_money'],
outputCol='features')
labeled_df = df_assembler.transform(df)

In [4]:
labeled_df = labeled_df["is_default","features"]

In [5]:
data_set = labeled_df.select(['features', 'is_default'])

In [6]:
train_df, test_df = data_set.randomSplit([0.8, 0.2])

In [7]:
log_reg = LogisticRegression(labelCol = 'is_default').fit(train_df)

In [8]:
test_result = log_reg.evaluate(test_df).predictions

/usr/local/spark/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [9]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='is_default')
print('AUC：', evaluator.evaluate(test_result))

AUC： 0.8014095445033663


In [14]:
print(MulticlassClassificationEvaluator(labelCol="is_default",predictionCol="prediction", metricName="f1").evaluate(test_result))

0.7877607356910269


better lr

In [10]:
for i in range(0,40):
    threshold = 0.3+0.01*i
    log_reg_2 = LogisticRegression(threshold = threshold,labelCol = 'is_default').fit(train_df)
    test_result_2 = log_reg_2.evaluate(test_df).predictions
    evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='is_default')
    print('AUC：'+str(threshold), evaluator.evaluate(test_result_2))

AUC：0.3 0.8014114612612473


AUC：0.31 0.8014110444230913


AUC：0.32 0.8014116347984709


AUC：0.32999999999999996 0.8014071114940454


AUC：0.33999999999999997 0.8014132951747241


...

AUC：0.65 0.801412768458728


AUC：0.6599999999999999 0.8014164999450664


AUC：0.6699999999999999 0.8014135907984373


AUC：0.6799999999999999 0.8014130605942559


AUC：0.69 0.8014117019460404


In [21]:
log_reg_2 = LogisticRegression(threshold = 0.41,labelCol = 'is_default').fit(train_df)
test_result_2 = log_reg_2.evaluate(test_df).predictions
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='is_default')
print('AUC：', evaluator.evaluate(test_result_2))

AUC： 0.8006012437350225


dt

In [7]:
dt = DecisionTreeClassifier(labelCol="is_default", featuresCol="features",maxDepth=15)

In [8]:
dt_model = dt.fit(train_df)

21/12/19 14:07:32 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
dt_pred = dt_model.transform(test_df)

In [10]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='is_default')
print('AUC：', evaluator.evaluate(dt_pred))

AUC： 0.7556666089317783


rf

In [14]:
numTrees_list = [20,50,100,150,200,300]
maxDepth_list = [5,10,15,20,30,40,50]
impurity_list = ['gini', 'entropy']
maxBins_list = [24,32,40]
for numTrees in numTrees_list:
    for maxDepth in maxDepth_list:
        for impurity in impurity_list:
            for maxBins in maxBins_list:
                rf_classifier=RandomForestClassifier(labelCol='is_default', featuresCol="features",numTrees=numTrees,maxDepth=maxDepth,impurity=impurity,maxBins=maxBins).fit(train_df)  
                rf_predictions=rf_classifier.transform(test_df)
                evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='is_default')
                print('AUC：'+str(numTrees)+str(maxDepth)+impurity+str(maxBins), evaluator.evaluate(rf_predictions))

AUC：205gini24 0.8194834321545822


AUC：205gini32 0.8185329671994049


AUC：205gini40 0.8212252118992234


AUC：205entropy24 0.8110912137195835


AUC：205entropy32 0.8015992903530812


...

AUC：2015entropy32 0.8607346375257043


21/12/18 18:49:31 WARN DAGScheduler: Broadcasting large task binary with size 1473.4 KiB
21/12/18 18:49:32 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB
21/12/18 18:49:35 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
21/12/18 18:49:39 WARN DAGScheduler: Broadcasting large task binary with size 5.8 MiB
21/12/18 18:49:41 WARN DAGScheduler: Broadcasting large task binary with size 1056.9 KiB
21/12/18 18:49:43 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
21/12/18 18:49:46 WARN DAGScheduler: Broadcasting large task binary with size 1438.5 KiB
21/12/18 18:49:48 WARN DAGScheduler: Broadcasting large task binary with size 12.6 MiB
21/12/18 18:49:52 WARN DAGScheduler: Broadcasting large task binary with size 1858.0 KiB
21/12/18 18:49:54 WARN DAGScheduler: Broadcasting large task binary with size 7.5 MiB


AUC：2015entropy40 0.860563475797644


21/12/18 18:50:13 WARN DAGScheduler: Broadcasting large task binary with size 1028.0 KiB
21/12/18 18:50:14 WARN DAGScheduler: Broadcasting large task binary with size 1721.8 KiB
21/12/18 18:50:16 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
21/12/18 18:50:19 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
21/12/18 18:50:22 WARN DAGScheduler: Broadcasting large task binary with size 7.0 MiB
21/12/18 18:50:24 WARN DAGScheduler: Broadcasting large task binary with size 1290.1 KiB
21/12/18 18:50:26 WARN DAGScheduler: Broadcasting large task binary with size 10.5 MiB
21/12/18 18:50:29 WARN DAGScheduler: Broadcasting large task binary with size 1766.3 KiB
21/12/18 18:50:31 WARN DAGScheduler: Broadcasting large task binary with size 15.3 MiB
21/12/18 18:50:35 WARN DAGScheduler: Broadcasting large task binary with size 2.3 MiB
21/12/18 18:50:38 WARN DAGScheduler: Broadcasting large task binary with size 21.5 MiB
21/12/18 18:50:43 WARN DAGScheduler: Br

Py4JJavaError: An error occurred while calling o3380.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 787.0 failed 1 times, most recent failure: Lost task 0.0 in stage 787.0 (TID 1289) (192.168.0.105 executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.ml.tree.impl.DTStatsAggregator$$Lambda$3574/1330471226.get$Lambda(Unknown Source)
	at java.lang.invoke.LambdaForm$DMH/245672235.invokeStatic_L_L(LambdaForm$DMH)
	at java.lang.invoke.LambdaForm$MH/2068562923.linkToTargetMethod(LambdaForm$MH)
	at org.apache.spark.ml.tree.impl.DTStatsAggregator.<init>(DTStatsAggregator.scala:54)
	at org.apache.spark.ml.tree.impl.RandomForest$.$anonfun$findBestSplits$22(RandomForest.scala:651)
	at org.apache.spark.ml.tree.impl.RandomForest$.$anonfun$findBestSplits$22$adapted(RandomForest.scala:647)
	at org.apache.spark.ml.tree.impl.RandomForest$$$Lambda$3571/1318815440.apply(Unknown Source)
	at scala.Array$.tabulate(Array.scala

In [11]:
rf_classifier=RandomForestClassifier(labelCol='is_default', featuresCol="features",numTrees=20,maxDepth=5,impurity='entropy').fit(train_df)  

In [12]:
rf_predictions=rf_classifier.transform(test_df)

In [13]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='is_default')
print('AUC：', evaluator.evaluate(rf_predictions))

AUC： 0.8015992903530811


gbt

In [15]:
gbt_classifier = GBTClassifier(labelCol='is_default', featuresCol="features", maxIter=15).fit(train_df)

In [16]:
gbt_predictions=gbt_classifier.transform(test_df)

In [17]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='is_default')
print('AUC：', evaluator.evaluate(rf_predictions))

AUC： 0.8131785792222486


MLPC

In [7]:
layers = [38,50,2]

In [8]:
MLPC_trainer = MultilayerPerceptronClassifier(labelCol='is_default', featuresCol="features", maxIter=100, layers=layers, blockSize=128, seed=1234)

In [9]:
MLPC_model = MLPC_trainer.fit(train_df)

21/12/19 17:05:56 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/12/19 17:05:56 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [10]:
MLPC_predictions = MLPC_model.transform(test_df)

In [11]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='rawPrediction',labelCol='is_default')
print('AUC：', evaluator.evaluate(MLPC_predictions))

AUC： 0.641615563116166
